In [2]:
import pandas as pd
import os
import requests
import time
from tqdm.auto import tqdm


keywords = pd.read_csv(r"C:\Users\besff\OneDrive\Documents\GitHub\datagenerator\datagenerator\data\keywords.csv")

c:\Users\besff\OneDrive\Documents\GitHub\datagenerator\datagenerator\url\data


In [67]:

stellenanzeigende = {
    "url": "https://jobs.meinestadt.de/?service=jobsSearch",
    "headers":{
        "Authority": "jobs.meinestadt.de",
        "Method": "POST",
        "Path": "/?service=jobsSearch",
        "Scheme": "https",
        "Accept": "application/json",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en-GB;q=0.9,en;q=0.8",
        "Content-Type": "application/x-www-form-urlencoded",
        "Origin": "https://jobs.meinestadt.de",
        "Sec-Ch-Ua": "\"Google Chrome\";v=\"113\", \"Chromium\";v=\"113\", \"Not-A.Brand\";v=\"24\"",
        "Sec-Ch-Ua-Mobile": "?0",
        "Sec-Ch-Ua-Platform": "\"Windows\"",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
        },  
    "data":{
        "jobApprenticeshipType": "stelle",
        "location": "deutschland",
        "radius": 1000,
        "words": '{keyword}',
        "query": '{keyword}',
        "categoryType": "jk",
        "offset": 0,
        "count": 20,
        "showDeactivated": False,
        "minijob": False,
        "filtersWithFacetCount": {},
        "order": "search(stelle,true)",
        "requestSource": "SEARCH",
        "relatedSeoTags": False,
        "distantRelatedCount": 0,
        "page": '{paginaton}',
        "maxPages": 50,
        "vacanciesMode": False,
        "userModified": True
    },
} 

df = pd.DataFrame()
df["keyword"] = keywords["general"].dropna().tolist()
df["category"] = "general"
for key, value in meinestadtde.items():
    df[key] = str(value)
df

,keyword,category,url,headers,data
0,kurier,general,https://jobs.meinestadt.de/?service=jobsSearch,"{'Authority': 'jobs.meinestadt.de', 'Method': ...","{'jobApprenticeshipType': 'stelle', 'location'..."
1,pflegehelfer,general,https://jobs.meinestadt.de/?service=jobsSearch,"{'Authority': 'jobs.meinestadt.de', 'Method': ...","{'jobApprenticeshipType': 'stelle', 'location'..."
2,versandabwicklung,general,https://jobs.meinestadt.de/?service=jobsSearch,"{'Authority': 'jobs.meinestadt.de', 'Method': ...","{'jobApprenticeshipType': 'stelle', 'location'..."
3,aushilfe,general,https://jobs.meinestadt.de/?service=jobsSearch,"{'Authority': 'jobs.meinestadt.de', 'Method': ...","{'jobApprenticeshipType': 'stelle', 'location'..."
4,abräumer,general,https://jobs.meinestadt.de/?service=jobsSearch,"{'Authority': 'jobs.meinestadt.de', 'Method': ...","{'jobApprenticeshipType': 'stelle', 'location'..."
...,...,...,...,...,...
200,mechatroniker,general,https://jobs.meinestadt.de/?service=jobsSearch,"{'Authority': 'jobs.meinestadt.de', 'Method': ...","{'jobApprenticeshipType': 'stelle', 'location'..."
201,ausbaufacharbeiter,general,https://jobs.meinestadt.de/?service=jobsSearch,"{'Authority': 'jobs.meinestadt.de', 'Method': ...","{'jobApprenticeshipType': 'stelle', 'location'..."
202,glaser,general,https://jobs.meinestadt.de/?service=jobsSearch,"{'Authority': 'jobs.meinestadt.de', 'Method': ...","{'jobApprenticeshipType': 'stelle', 'location'..."
203,estrichleger,general,https://jobs.meinestadt.de/?service=jobsSearch,"{'Authority': 'jobs.meinestadt.de', 'Method': ...","{'jobApprenticeshipType': 'stelle', 'location'..."


In [68]:

df["data"] = df.apply(lambda row: row["data"].replace("{keyword}", row["keyword"]), axis=1)

# list_columns = df.columns[df.applymap(lambda x: isinstance(x, list)).any()].tolist()
# df = df.explode(list_columns)
   


In [69]:
def pagination(row):
    try:
        time.sleep(2)
        data = eval(row["data"].replace("{paginaton}", '2'))
        headers = eval(row["headers"])
        response  = requests.post(url="https://jobs.meinestadt.de/?service=jobsSearch",headers=headers, data=data)
        end = round(float(response.json()["results"]["totalPages"]))
        return [i for i in range(1, end)]
    except:
        return None


tqdm.pandas()
df["pagination"] = df.progress_apply(pagination, axis=1)

list_columns = df.columns[df.applymap(lambda x: isinstance(x, list)).any()].tolist()
df = df[df["pagination"].notna()]
df = df.explode(list_columns)

# df.to_csv(os.path.join(resource_path, "meinestadtde_urls.csv"), index=False)

100%|██████████| 205/205 [09:19<00:00,  2.73s/it]


In [30]:
def replace_pagination(row):
    try:
        data = eval(row["data"])
        data["page"] = row["pagination"]
        return str(data)
    except:
        return None


df = pd.read_csv(os.path.join(resource_path, "meinestadtde_urls.csv") )
df = df[df["pagination"].notna()]
df["pagination"] = df["pagination"].astype(int)
df["data"] = df.apply(replace_pagination, axis=1)
df.to_csv(os.path.join(resource_path, "meinestadtde_urls1.csv"), index=False)